## Train model and save it

This model predicts remission likelihood of breast cancer using the UCI breast cancer dataset

In [32]:
%run train.py

Accuracy: 68.42%


In [36]:
import enum
import os
import joblib

dir_path = '/Users/gkv/Startup/flighty-sample/model_dir/'
input_pipeline = joblib.load(os.path.join(dir_path, 'input_pipeline.joblib'))

output_pipeline = None
feature_names = input_pipeline[0].get_feature_names_out()
col_dict = {}

for feat in feature_names:
    val = feat.split('_', 1)
    if val[0] in col_dict:
        col_dict[val[0]].append(val[1])
    else:
        col_dict[val[0]] = [val[1]]

enums = [enum.Enum(k, {item: item for item in v}) for k, v in col_dict.items()]
enums = {key: value for key, value in zip(col_dict.keys(), map(lambda x: (x, ...), enums))}

## Import flighty SDK and initialize

We create an endpoint, which is the stable URl where the model will be hosted, an artifact which is a folder holding our model files, and a handler, which is the actual Docker image and Python code that will run the model.

In [26]:
import flighty as f

# f.set_hostname('127.0.0.1:8002')

In [36]:
f.create_endpoint(name='doc-rec')
f.create_artifact(path='./model_dir', name='xgboost', version=0)
f.create_handler(endpoint='doc-rec', name='xgboost', 
  docker_image='docker.io/gvashishtha/flighty:flighty-demo',
  code_path='./code_sample', artifacts=[{'name': 'xgboost', 'version': 0}])

{'name': 'doc-rec', 'url': 'http://127.0.0.1/doc-rec'}
{'name': 'xgboost',
 'path': '/code/flighty-files/xgboost/0',
 'type': 'model',
 'version': 0}
{'name': 'doc-rec-xgboost-0-code',
 'path': '/code/flighty-files/doc-rec-xgboost-0-code/0',
 'type': 'code',
 'version': 0}
{'docker_image': 'docker.io/gvashishtha/flighty:flighty-demo',
 'endpoint': 'doc-rec',
 'name': 'xgboost',
 'version': 0}


In [31]:
response = f.invoke(endpoint='doc-rec', body={"data": {
    "age": "40-49",
    "meno": "lt40",
    "tumor_size": "0-4",
    "inv_nodes": "0-2",
    "node_caps": "yes",
    "deg_malig": "1",
    "breast": "left",
    "breastquad": "left_up",
    "irradiat": "yes"
  }})
print(response.text, response.status_code)

"Your prediction is ['no-recurrence-events']" 200


In [32]:
f.list_handlers()

[{'docker_image': 'docker.io/gvashishtha/flighty:flighty-demo',
  'endpoint_id': 1,
  'id': 1,
  'name': 'xgboost',
  'prod_traffic': 100,
  'shadow_traffic': 0,
  'version': 0}]


## Clean up

In [38]:
f.delete_handler(endpoint='doc-rec', name='xgboost', version=0)
f.delete_endpoint(name='doc-rec')
f.delete_artifact(name='xgboost', version=0)

None
'Artifact doc-rec-xgboost-0-code with version 0 successfully deleted.'
'Endpoint doc-rec successfully deleted.'
'Artifact xgboost with version 0 successfully deleted.'


## Confirm our resources were deleted

In [39]:
f.list_artifacts()
f.list_endpoints()
f.list_handlers()

[]
[]
[]
